# Notebook Description

In this Notebook, we will uptrain our Neural Network on an increased number of Jobs.<br>
If this is the first iteration, we will uptrain the Supervised Neural Network to 9 Jobs.<br>
If he have uptrained it already before on more than 8 Jobs, we increase the maximum number of Jobs by one and continue training that uptrained version.

# Code

In [ ]:
import numpy as np
import os
import pickle
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, LSTM, Concatenate, LeakyReLU, Softmax, Dropout
from keras.layers import Lambda, Flatten, Bidirectional, TimeDistributed, Reshape, MultiHeadAttention, LayerNormalization
from keras.activations import tanh
from keras.models import Model, Sequential
import keras.backend
import random
from copy import copy
#import necessary notebooks
import import_ipynb
from Jobs_and_Machines import *
from States_and_Policies import *
from Action_Pointer import *

In [ ]:
#change working directory
work_path = input("Working directory to save the Neural Network.\n")
os.chdir(work_path)

### Create Estimated Training Data

Set the increased number of Jobs.

In [ ]:
n_estim = int(input("What is the new number of Jobs?\n"))

n = 8
n_min = 3

m = 4
m_min = 2

DS_max = 10
DS_min = 1

In [ ]:
def load_data(DS_min,DS_max):
    
    #dictionary will contain data from all respective data sets
    data_dict = dict(((n_state,m_state),[[],[]]) 
                           for n_state in range(n_min,n+1) for m_state in range(m_min,m+1))
    
    #load data
    for n_state in range(n_min, n+1):
        for m_state in range(m_min, m+1):
            #resource information of the jobs: processing time on every machine and its metadata
            x_res = []
            #urgency information of the jobs: earliness and weight
            x_urg = []
            #targets
            y = []
            #loop over every data set "DS"
            for DS in ["0"*(2-len(str(i)))+str(i) for i in range(DS_min,DS_max+1)]:
                training_data_path = f'Data/DataSet_{DS}/LSTM_Data_RR_{DS}/{n_state}-jobs-{m_state}-machines_{DS}.pickle'
                with open(training_data_path, 'rb') as f:
                    #load data set of key (n_state,m_state)
                    data = pickle.load(f)
                    #get data about its resources for every job of one state of one Job Scheduling Problem
                    jobs_resources = data[0][0][:,:,:-2].reshape((-1,n_state,m_state,4))
                    #get data abouts its urgenicies for every job  of one state of one Job Scheduling Problem
                    jobs_urgencies = data[0][0][:,:,-2:]
                    #add to data of states of other Job Scheduling Problems
                    x_res.append(jobs_resources)
                    x_urg.append(jobs_urgencies)
                    y.append(data[1][0])
            #transform list of data from states of Job Scheduling Problems to numpy array and add to final data dictionary
            data_dict[(n_state,m_state)][0].append(np.concatenate(x_res))
            data_dict[(n_state,m_state)][0].append(np.concatenate(x_urg))
            data_dict[(n_state,m_state)][1].append(np.concatenate(y))
            
    return data_dict

We will now load the estimated data and transform it into the form of training data.<br>
Note that the path in the function might have to be changed by user.

In [ ]:
#load estimated data
def load_estim_data():
    
    #dictionary will contain all estimated
    estim_data_dict = dict(((n_state,m_state),[[],[]]) 
                           for n_state in range(n+1, n_estim+1) for m_state in range(m_min,m+1))
    
    #load estimated data from n+1 to n_estim
    for n_state in range(n, n_estim+1):
        for m_state in range(m_min, m+1):
            #resource information of the jobs: processing time on every machine and its metadata
            x_res = []
            #urgency information of the jobs: earliness and weight
            x_urg = []
            #targets
            y = []
            """use your own path here"""
            estim_data_path = f'Data/EstimData/{n_state}_Jobs/LSTM_EstimData_RR/{n_state}-jobs-{m_state}-machines.pickle'
            with open(estim_data_path, 'rb') as f:
                data = pickle.load(f)
                #get data about its resources for every job of one state of one Job Scheduling Problem
                jobs_resources = data[0][0][:,:,:-2].reshape((-1,n_state,m_state,4))
                #get data abouts its urgenicies for every job  of one state of one Job Scheduling Problem
                jobs_urgencies = data[0][0][:,:,-2:]
                #add to data of states of other Job Scheduling Problems
                x_res.append(jobs_resources)
                x_urg.append(jobs_urgencies)
                y.append(data[1][0])

            #transform list of data from states of Job Scheduling Problems to numpy array and add to final data dictionary
            estim_data_dict[(n_state,m_state)][0].append(np.concatenate(x_res))
            estim_data_dict[(n_state,m_state)][0].append(np.concatenate(x_urg))
            estim_data_dict[(n_state,m_state)][1].append(np.concatenate(y))

    return estim_data_dict

Load the estimated data and bring it into the correct form by splitting it every <i>(n_state,m_state)</i>-subset from it into a list of inputs and a list of targets.

In [ ]:
estim_dict = load_estim_data()

In [ ]:
x_estim_list, y_estim_list = [], []
for key in estim_dict:
    
    #estimated data
    x_estim, y_estim = estim_dict[key]
    x_estim_list.append(x_estim)
    y_estim_list.append(y_estim)

Load Network.

In [ ]:
#load desired version of Neural Network
def load_NN(NN_name):
    """path might have to be updated based on users storage"""
    path = 'D:\\Job-Scheduling-Files\'
    
    NN = keras.models.load_model(f'{path}{NN_name}.h5', custom_objects={'FeedForward': FeedForward, 'Pointer': Pointer, 'MSE_with_Softmax': MSE_with_Softmax, 'costs':costs})
    NN.run_eagerly = True
    
    return NN

In [ ]:
#Uptrained_Neural_Network = load_NN(f"Final_Pointer3")
Uptrained_Neural_Network = load_NN(f"Uptrained_Final_Pointer_{n_estim-1}_Jobs")

### Compile Model

Compile the model the same way as before.

In [ ]:
learning_rate = 0.001

Neural_Network.compile(
        #custom loss
        loss = MSE_with_Softmax,
        #optimizer
        optimizer = keras.optimizers.Adam(learning_rate = learning_rate),
        run_eagerly=True,
        #custom metric
        metrics = [costs])

### Uptraining

We will now use the estimated data to uptrain the Neural Network.<br>
Again, one epoch will consist of the states related to one <i>(n_state,m_state)</i>-subset only.<br>
Since is has been trained for any number of Jobs greater than <i>n</i> but lower than <i>n_estim</i> already, we will first train it on the data of <i>n_estim</i> jobs. We loop 5 times through the data, resulting in 5*3 epochs.

In [ ]:
print(f'We train for {n_estim} Jobs')
ep=0*3
#load the last three subsets, as these are (n_estim,4), (n_estim,3) and (n_estim,2)
for j in range(len(x_estim_list)-3,len(x_estim_list)):
    #train one epoch on  every subset
    history = Uptrained_Neural_Network.fit(x_estim_list[j], y_estim_list[j], shuffle=True, batch_size=128, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
    ep = history.epoch[-1]+1

In [ ]:
print(f'We train for {n_estim} Jobs')
ep=1*3
#load the last three subsets, as these are (n_estim,4), (n_estim,3) and (n_estim,2)
for j in range(len(x_estim_list)-3,len(x_estim_list)):
    #train one epoch on  every subset
    history = Uptrained_Neural_Network.fit(x_estim_list[j], y_estim_list[j], shuffle=True, batch_size=128, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
    ep = history.epoch[-1]+1

In [ ]:
print(f'We train for {n_estim} Jobs')
ep=2*3
#load the last three subsets, as these are (n_estim,4), (n_estim,3) and (n_estim,2)
for j in range(len(x_estim_list)-3,len(x_estim_list)):
    #train one epoch on  every subset
    history = Uptrained_Neural_Network.fit(x_estim_list[j], y_estim_list[j], shuffle=True, batch_size=128, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
    ep = history.epoch[-1]+1

In [ ]:
print(f'We train for {n_estim} Jobs')
ep=3*3
#load the last three subsets, as these are (n_estim,4), (n_estim,3) and (n_estim,2)
for j in range(len(x_estim_list)-3,len(x_estim_list)):
    #train one epoch on  every subset
    history = Uptrained_Neural_Network.fit(x_estim_list[j], y_estim_list[j], shuffle=True, batch_size=128, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
    ep = history.epoch[-1]+1

In [ ]:
print(f'We train for {n_estim} Jobs')
ep=4*3
#load the last three subsets, as these are (n_estim,4), (n_estim,3) and (n_estim,2)
for j in range(len(x_estim_list)-3,len(x_estim_list)):
    #train one epoch on  every subset
    history = Uptrained_Neural_Network.fit(x_estim_list[j], y_estim_list[j], shuffle=True, batch_size=128, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
    ep = history.epoch[-1]+1

If <i>n_estim > 9</i>, we retrain the Network on the entire estimated data set. One loop therefore consists of 3*i epochs.<br>
We do <i>n_estim-n+1</i> loops.

In [ ]:
print(f'We train for {n_estim} Jobs')
ep=5*3
for j in range(len(x_estim_list)):
    #train one epoch on  every subset
    history = Uptrained_Neural_Network.fit(x_estim_list[j], y_estim_list[j], shuffle=True, batch_size=128, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
    ep = history.epoch[-1]+1

In [ ]:
print(f'We train for {n_estim} Jobs')
ep=5*3 + len(x_estim_list)
for j in range(len(x_estim_list)):
    #train one epoch on  every subset
    history = Uptrained_Neural_Network.fit(x_estim_list[j], y_estim_list[j], shuffle=True, batch_size=128, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
    ep = history.epoch[-1]+1

In [ ]:
print(f'We train for {n_estim} Jobs')
ep=5*3 + 2*len(x_estim_list)
for j in range(len(x_estim_list)):
    #train one epoch on  every subset
    history = Uptrained_Neural_Network.fit(x_estim_list[j], y_estim_list[j], shuffle=True, batch_size=128, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
    ep = history.epoch[-1]+1

In [ ]:
print(f'We train for {n_estim} Jobs')
ep=5*3 + 3*len(x_estim_list)
for j in range(len(x_estim_list)):
    #train one epoch on  every subset
    history = Uptrained_Neural_Network.fit(x_estim_list[j], y_estim_list[j], shuffle=True, batch_size=128, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
    ep = history.epoch[-1]+1

In [ ]:
print(f'We train for {n_estim} Jobs')
ep=5*3 + 4*len(x_estim_list)
for j in range(len(x_estim_list)):
    #train one epoch on  every subset
    history = Uptrained_Neural_Network.fit(x_estim_list[j], y_estim_list[j], shuffle=True, batch_size=128, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
    ep = history.epoch[-1]+1

Finally, we save the uptrained version of the Neural Network.

In [ ]:
Uptrained_Neural_Network.save(f'Uptrained_Neural_Network_{n_estim}_Jobs.h5')